In [77]:
import pandas as pd
import numpy as np

# Data processing

In [81]:
data = pd.read_csv('./data.csv')
data.head()
data.set_index(pd.to_datetime(data.date), inplace=True)
data.head()

,ticker,date,last,volume
date,,,,
2013-01-04,1332 JT,2013-01-04,169.0987,1464100
2013-01-07,1332 JT,2013-01-07,166.3266,1783500
2013-01-08,1332 JT,2013-01-08,166.3266,1759800
2013-01-09,1332 JT,2013-01-09,165.4026,767800
2013-01-10,1332 JT,2013-01-10,167.2507,1503100


In [83]:
df_last = pd.pivot_table(data, values='last', index=data.index, columns='ticker')
df_volume = pd.pivot_table(data, values='volume', index=data.index, columns='ticker')
df_ret = last.pct_change()
print(df_ret.shape)

(2005, 248)


After the data processing, we can see that the data.csv consists 2005 days and 248 stocks.

So I want to construct cross-section features and then make the trading strategies 

My plan is:
1. Single alpha backtesting analysis
2. Construct alphas based on `data.csv`
3. Use these alphas to predict returns via some methods such as LR, Lasso, Ridge as well as using cross validation
4. Portfolio Optimization

# Alpha Analysis

In [95]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import scipy.stats
import concurrent
rc = {'axes.facecolor': 'white',
      'savefig.facecolor': 'white'}
mpl.rcParams.update(rc)
mpl.rcParams['axes.unicode_minus']=False
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['figure.dpi'] = 200
plt.style.use('_mpl-gallery')

class FactorAnalysis:
    def __init__(self, factor, ret, list_periods, n_group,benchmark = False):
        self.factor = factor
        self.ret = ret
        self.list_periods = list_periods
        self.n_group = n_group
        self.rf = 0.02
        self.benchmark = benchmark
    
    def calc_results(self, adj_periods):
            adj_date = []
            ret_results = pd.DataFrame()
            excess_ret_results = pd.DataFrame()
            ex_results = pd.DataFrame()
            ic_list = []
            rank_ic_list = []
            df_last = pd.DataFrame()
            df_group_num = pd.DataFrame()
            group_ic_list = []
            df_group_inner_ic = pd.DataFrame()

            for adj_idx in range(0, len(self.factor), adj_periods):
                adj_day = self.factor.index[adj_idx]
                adj_date.append(adj_day)
                df_1 = self.factor.iloc[adj_idx, :].to_frame()
                df_1.columns = ['factor']
                df_1 = pd.concat([df_1, self.ret.iloc[adj_idx: adj_idx+adj_periods, :].T],
                                 axis=1)
                df_1.dropna(subset=['factor'], inplace=True)
                df_1['factor'] += np.random.normal(0, 1, size=(df_1.shape[0])) * 1e-12
                df_1.dropna(subset=[df_1.columns[1]], inplace=True)
                df_1.iloc[:, 2:] = df_1.iloc[:, 2:].fillna(0)
                df_1['ret'] = np.cumprod(1 + df_1.iloc[:, 1:], axis=1).iloc[:, -1] - 1

                df_2 = df_1.loc[:, ['factor', 'ret']].copy()
                df_2['group'] = pd.qcut(df_2['factor'], self.n_group, labels=list(range(1, self.n_group+1)))

                df_group_num_tmp = df_2['group'].value_counts().to_frame()
                df_group_num_tmp.columns = [adj_day]
                df_group_num = pd.concat([df_group_num, df_group_num_tmp], axis=1)

                df_ret = df_2.groupby(by=['group'])['ret'].mean().to_frame()  

                group_ic = scipy.stats.spearmanr(df_ret.index, df_ret['ret'])[0]
                group_ic_list.append(group_ic)

                df_ret.columns = [adj_day]
                ret_results = pd.concat([ret_results, df_ret], axis=1)

                ic = df_1.loc[:, ['factor', 'ret']].corr(method='pearson').iloc[0, 1]
                rank_ic = df_1.loc[:, ['factor', 'ret']].corr(method='spearman').iloc[0, 1]
                ic_list.append(ic)
                rank_ic_list.append(rank_ic)

                if len(df_last) == 0:
                    df_exchange = pd.DataFrame(data=[0]*self.n_group,
                                               index=list(np.arange(1, self.n_group+1)),
                                               columns=[adj_day])
                else:
                    ex_list = []
                    for i in np.arange(1, self.n_group+1):
                        stocks_now = df_2[df_2['group'] == i].index
                        stocks_last = df_last[df_last['group'] == i].index
                        retain_rate = len(set(stocks_last).intersection(set(stocks_now))) / len(stocks_last)
                        exchange_rate = 1 - retain_rate
                        ex_list.append(exchange_rate)
                    df_exchange = pd.DataFrame(data=ex_list,
                                               index=list(np.arange(1, self.n_group+1)),
                                               columns=[adj_day])
                ex_results = pd.concat([ex_results, df_exchange], axis=1)
                df_last = df_2.copy()

            df_ic = pd.DataFrame(data=[ic_list, rank_ic_list],
                                       index=['ic', 'rank_ic'],
                                       columns=adj_date).T

            df_group_ic = pd.DataFrame(data=group_ic_list,
                                       columns=['group_ic'],
                                       index=adj_date)

            results = {
                'adj_period': adj_periods,
                'ic': df_ic,
                'group_ic': df_group_ic,
                'group_inner_ic': df_group_inner_ic,
                'ret': ret_results,
                'group_num': df_group_num,
                'exchange': ex_results
            }

            return results

    def calc_multiple_periods(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = [executor.submit(self.calc_results, i)
                       for i in self.list_periods]
        results = [f.result() for f in concurrent.futures.as_completed(results)]
        results = sorted(results, key = lambda e: e.__getitem__('adj_period'))  # 按照adj_period排序
        self.results = results
        return results

    def summary_ic_table(self, start=False, end=False, kind='rank_ic'):
        if not start:
            start = self.factor.index[0]
        if not end:
            end = self.factor.index[-1]
        summary_ic = pd.DataFrame()
        summary_group_ic = pd.DataFrame()
        for result in self.results:
            adj_period = result['adj_period']
            ic = result['ic'].copy()
            ic = ic[(ic.index >= start) & (ic.index <= end)]
            ic_mean = ic.mean()
            ic_std = ic.std(ddof=1)
            ic_ir = np.abs(ic_mean) / ic_std
            ic_skew = ic.skew()
            ic_kurtosis = ic.kurtosis()
            t_0 = np.sqrt(len(ic)) * ic_mean / ic_std
            p_0 = np.abs(t_0).apply(lambda x: (1-scipy.stats.t.cdf(x, len(ic)-1)) / 2)
            df_tmp1 = pd.DataFrame(data=[ic_mean, ic_std,ic_ir,
                                 ic_skew, ic_kurtosis,
                                 t_0, p_0],
                                   index=['IC Mean','IC Std', 'IC IR',
                                          'IC Skew', 'IC Kurtosis',
                                          't: IC!=0', 'p: IC!=0'])
            df_tmp1.columns = [[f'period_{adj_period}', f'period_{adj_period}'],
                               ['IC', 'rank IC']]
            df_tmp1 = df_tmp1.round(4)
            summary_ic = pd.concat([summary_ic, df_tmp1], axis=1)
            group_ic = result['group_ic'].copy()
            group_ic = group_ic[(group_ic.index >= start) & (group_ic.index <= end)]
            group_ic_mean = group_ic.mean()
            group_ic_std = group_ic.std(ddof=1)
            group_ic_ir = group_ic_mean / group_ic_std
            group_ic_skew = group_ic.skew()
            group_ic_kurtosis = group_ic.kurtosis()
            t_0 = group_ic_ir * np.sqrt(len(group_ic))
            p_0 = np.abs(t_0).apply(lambda x: (1-scipy.stats.t.cdf(x, len(ic)-1)) / 2)
            df_tmp2 = pd.DataFrame(data=[group_ic_mean, group_ic_std, group_ic_ir,
                                         group_ic_skew, group_ic_kurtosis,
                                         t_0, p_0],
                                   index=['IC Mean', 'IC Std', 'IC IR', 'IC Skew', 'IC Kurtosis',
                                          't: IC!=0', 'p: IC!=0'])
            df_tmp2.columns = [f'period_{adj_period}: group rank IC']
            df_tmp2 = df_tmp2.round(4)
            summary_group_ic = pd.concat([summary_group_ic, df_tmp2], axis=1)
        dict1 = {}
        if kind == 'rank_ic':
            df_ic = summary_ic.iloc[:, list(np.arange(0, 2*len(self.results), 2)+1)]
            df_ic.columns = [f'{x[0]}: {x[1]}' for x in df_ic.columns]
            dict1['ic'] = df_ic
            dict1['group_ic'] = summary_group_ic
            return dict1
        elif kind == 'ic':
            df_ic = summary_ic.iloc[:, list(np.arange(0, 2*len(self.results), 2))]
            df_ic.columns = [f'{x[0]}: {x[1]}' for x in df_ic.columns]
            dict1['ic'] = df_ic
            dict1['group_ic'] = summary_group_ic
            return dict1

    def plot_cum_ic(self,  kind='rank_ic'):
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ic = self.results[i]['ic'][kind].to_frame()
            ic.index = pd.to_datetime(ic.index)
            ic.sort_index(ascending=True, axis=1, inplace=True)
            ic['cum'] = np.cumsum(ic[kind])

            ax = fig.add_subplot(100*(len(self.results)) + 10 + (i+1))
            ax.plot(ic.index, ic['cum'], alpha=1, color='C0')
            ax.set_title(f'Adj_periods_{adj_period}: {kind}', fontsize=15)
            ax.legend(['cumulative'], loc='upper left')
        plt.tight_layout()
        plt.show()

    def plot_ic_bar(self, frequency='Y', kind='rank_ic'):
        df_ic = pd.DataFrame()
        for result in self.results:
            ic = result['ic'].loc[:, kind].copy()
            ic = ic.resample(frequency, axis=0, label='right',
                             closed='right', kind='period').mean().to_frame()
            ic = (100 * ic).round(2)
            ic.columns = [result['adj_period']]
            df_ic = pd.concat([df_ic, ic], axis=1)
        df_ic.plot(kind='bar', figsize=(10, 5)) 
        plt.title('IC (%)', fontsize=15)
        plt.legend(fontsize=15)
        plt.show()

    def plot_daily_ic(self, kind='rank_ic'):
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ic = self.results[i]['ic'][kind].to_frame()
            ic = (100 * ic).round(2)
            ic['30d'] = ic[kind].rolling(window=int(20//adj_period)+1).mean()
            ic.index = pd.to_datetime(ic.index)
            ic.sort_index(ascending=True, axis=1, inplace=True)
            ax = fig.add_subplot(100*(len(self.results)) + 10 + (i+1))
            ax.bar(ic.index, ic[kind], alpha=0.6, width=2)
            ax.plot(ic.index, ic['30d'], alpha=1, color='red')
            ax.set_title(f'Adj_periods_{adj_period}: {kind} (%)', fontsize=15)
            ax.legend(['30d', 'daily'], loc='upper left')  # 先写plot的，再写bar的
            ax.axhline(3,color='black',linestyle='--')
            ax.axhline(-3,color='black',linestyle='--')
            ax.fill_between(ic.index, 3, -3,color='yellow',alpha=0.4)
            ax.set_ylim(-40, 40)
        plt.tight_layout()
        plt.show()

    def plot_monthly_ic(self, kind='rank_ic'):
        plt.style.use('default')
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ic = self.results[i]['ic'][kind].to_frame()
            ic = ic.resample('M', axis=0, label='right',
                             closed='right', kind='period').mean()
            ic.columns = [kind]
            ic['year'] = ic.index.year
            ic['month'] = ic.index.month
            ic = ic.pivot_table(index=['year'], columns=['month'],
                                values=[kind])
            ic.columns = [x[1] for x in ic.columns]
            ic = (100 * ic).round(2)
            ax = fig.add_subplot(100*len(self.results) + 10 + (i+1))
            ax.imshow(ic.values, cmap="summer")
            ax.set_xticks(np.arange(len(ic.columns)), labels=ic.columns, fontsize=15)
            ax.set_yticks(np.arange(len(ic.index)), labels=ic.index, fontsize=15)
            for j in range(len(ic.index)):
                for k in range(len(ic.columns)):
                    text = ax.text(k, j,ic.values[j, k],
                                        ha="center", va="center", color="black")
            ax.set_title(f'Adj_period_{adj_period}: Monthly IC (%)')
        plt.tight_layout()
        plt.show()
        plt.style.use('_mpl-gallery')

    def plot_monthly_ret(self):
        plt.style.use('default')
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ret = self.results[i]['ret'].T.copy()
            ret = (ret.iloc[:, -1] - ret.iloc[:, 0]).to_frame()  
            ret = np.log(1+ret)  
            ret = ret.resample('M', axis=0, label='right',
                               closed='right', kind='period').sum()
            ret = np.exp(ret) - 1  
            ret.columns = ['ret']
            ret['year'] = ret.index.year
            ret['month'] = ret.index.month
            ret = ret.pivot_table(index=['year'], columns=['month'],
                                  values=['ret'])
            ret.columns = [x[1] for x in ret.columns]
            ret = (100 * ret).round(2)
            ax = fig.add_subplot(100*len(self.results) + 10 + (i+1))
            ax.imshow(ret.values, cmap="summer") 
            ax.set_xticks(np.arange(len(ret.columns)), labels=ret.columns, fontsize=15)
            ax.set_yticks(np.arange(len(ret.index)), labels=ret.index, fontsize=15)
            for j in range(len(ret.index)):
                for k in range(len(ret.columns)):
                    text = ax.text(k, j,ret.values[j, k],
                                        ha="center", va="center", color="black")
            ax.set_title(f'Adj_period_{adj_period}: Long Short Monthly Return (%)')

        plt.tight_layout()
        plt.show()
        plt.style.use('_mpl-gallery')

    def summary_layer_ret(self,excess=False,long=True):
        def prod(list_1):
            return np.cumprod(list_1)[-1]

        summary_layer_ret_table = pd.DataFrame()
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            rf = self.rf
            ret = self.results[i]['ret'].T.copy()
            ret.index = pd.to_datetime(ret.index)

            if excess:
                if type(self.benchmark) == bool:
                    ret = ret - ret.mean(axis=1).values.reshape(len(ret), 1)
                else:
                    benchmark = self.benchmark + 1
                    benchmark['new_ret'] = benchmark.rolling(window=adj_period ).apply(prod)
                    benchmark = benchmark.loc[ret.index,'new_ret'] - 1
                    benchmark = benchmark.fillna(0)
                    ret = ret - benchmark.values.reshape(len(ret), 1)
            ret = ret * (long * 2 - 1)
            s_ret = ret.iloc[:, [0,-1]]
            for j in s_ret.columns:
                ret = s_ret.loc[:,j]
                AVolatility = np.std(ret)*np.sqrt(252/adj_period)
                WinningRatio = len(ret[ret > 0])/len(ret[ret != 0])
                PnLRatio = np.mean(ret[ret > 0]) / abs(np.mean(ret[ret < 0]))
                ret = np.cumprod(1 + ret, axis=0)
                AReturnRate = (ret[-1]/ret[0]) ** (1/(len(ret)*adj_period/252)) - 1
                SharpeRatio = (AReturnRate-rf)/AVolatility

                ret = ret.to_list()
                low_point = np.argmax((np.maximum.accumulate(ret)- ret)/np.maximum.accumulate(ret))
                if low_point == 0:
                    MaxDrawdown = 0
                high_point = np.argmax(ret[:low_point])
                MaxDrawdown = (ret[high_point] - ret[low_point]) / ret[high_point]

                df_tmp = pd.DataFrame(data=[AReturnRate, AVolatility,
                                            SharpeRatio, MaxDrawdown,
                                            WinningRatio, PnLRatio],
                                      index=['AReturnRate', 'AVolatility',
                                             'SharpeRatio', 'MaxDrawdown',
                                             'WinningRatio', 'PnLRatio'],
                                      columns = [f'period{adj_period } group{j}'])
                summary_layer_ret_table = pd.concat([summary_layer_ret_table, df_tmp], axis=1)
        return summary_layer_ret_table

    def plot_layer_ret_bar(self, excess=True, long=True):
        df_ret = pd.DataFrame()

        def prod(list_1):
            return np.cumprod(list_1)[-1]

        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ret = self.results[i]['ret'].T.copy()
            ret.columns = list(range(1, self.n_group+1))
            ret.index = pd.to_datetime(ret.index)

            if excess:
                if type(self.benchmark) == bool:
                    ret = ret - ret.mean(axis=1).values.reshape(len(ret), 1)
                else:
                    benchmark = self.benchmark + 1
                    benchmark['new_ret'] = benchmark.rolling(window=adj_period ).apply(prod)
                    benchmark = benchmark.loc[ret.index,'new_ret'] - 1
                    benchmark = benchmark.fillna(0)
                    ret = ret - benchmark.values.reshape(len(ret), 1)
            ret = np.cumprod(1 + ret * (long * 2 - 1), axis=0) - 1
            ret = (1 + ret) ** (1 / (len(ret) * adj_period)) - 1
            ret = 10000 * ret.iloc[-1, :].to_frame()
            ret.columns = [adj_period]
            df_ret = pd.concat([df_ret, ret], axis=1)
        df_ret.plot(kind='bar', figsize=(10, 5))
        plt.title('Group Return', fontsize=15)
        plt.ylabel('daily return (bps)', fontsize=15)
        plt.legend(fontsize=15)
        plt.show()

    def layer_ret_df(self):
        self.calc_multiple_periods()

        df_ret = pd.DataFrame()
        for i in range(len(self.results)):
            ret = self.results[i]['ret'].T.copy()
            ret.columns = list(range(1, self.n_group+1))
            ret.index = pd.to_datetime(ret.index)
            df_ret = pd.concat([df_ret, ret], axis=1)
        return df_ret

    def plot_layer_ret(self, excess=True, long=True):
        def prod(list_1):
            return np.cumprod(list_1)[-1]

        cmap = cm.get_cmap("RdYlGn")
        cmap = cmap(np.linspace(0, 1, self.n_group))
        fig = plt.figure(figsize=(10, 5*len(self.results)))

        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ret = self.results[i]['ret'].T.copy()
            ret.index = pd.to_datetime(ret.index)

            if excess:
                if type(self.benchmark) == bool:
                    ret = ret - ret.mean(axis=1).values.reshape(len(ret), 1)
                else:
                    benchmark = self.benchmark + 1
                    benchmark['new_ret'] = benchmark.rolling(window=adj_period ).apply(prod)
                    benchmark = benchmark.loc[ret.index,'new_ret'] - 1
                    benchmark = benchmark.fillna(0)
                    ret = ret - benchmark.values.reshape(len(ret), 1)
            ret = np.cumprod(1 + ret * (long * 2 - 1), axis=0)
            ax = fig.add_subplot(100*len(self.results) + 10 + (i+1))
            for j in range(len(ret.columns)):
                ax.plot(ret.index, ret.iloc[:, j].values, color=cmap[j], alpha=1)
            ax.legend(ret.columns, loc='upper left', fontsize=10)
            adj_period = self.results[i]['adj_period']
            ax.set_title(f'Adj_periods_{adj_period}: Net Value for Groups', fontsize=15)
            ax.axhline(1, linestyle='--', c='grey')
        plt.tight_layout()
        plt.show()

    def plot_long_short_ret(self):
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            ret = self.results[i]['ret'].T.copy()
            ret = np.cumprod(1 + (ret.iloc[:, -1] - ret.iloc[:, 0]), axis=0).to_frame(name='ret')
            ret['ADD'] = -(np.maximum.accumulate(ret.ret)- ret.ret)/np.maximum.accumulate(ret.ret)

            ax1 = fig.add_subplot(100*len(self.results) + 10 + (i+1))
            ax1.plot(ret.index, ret.loc[:, 'ret'], color='C0',alpha=1)
            ax1.legend(['Net_Value'], loc='upper left', fontsize=10)
            ax1.axhline(1, linestyle='--', c='grey')
            ax2 = ax1.twinx()
            ax2.fill_between(ret.index, 0,ret.loc[:, 'ADD'], color='red',alpha=0.5)
            ax2.set_ylim(-0.5, 0)
            ax2.legend(['Accumulated_Drawdown'], loc='lower right', fontsize=10)
            adj_period = self.results[i]['adj_period']
            ax2.set_title(f'Adj_periods_{adj_period}: Long Short Portfolio Net Value', fontsize=15)
            ax2.grid()

        plt.tight_layout()
        plt.show()

    def summary_long_short_ret(self):
        summary_long_short_table = pd.DataFrame()
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            rf = self.rf
            ret = self.results[i]['ret'].T.copy()
            ret = ret.iloc[:, -1] - ret.iloc[:, 0]
            AVolatility = np.std(ret)*np.sqrt(252/adj_period)
            WinningRatio = len(ret[ret > 0])/len(ret[ret != 0])
            PnLRatio = np.mean(ret[ret > 0]) / abs(np.mean(ret[ret < 0]))
            ret = np.cumprod(1 + ret, axis=0)
            AReturnRate = (ret[-1]/ret[0]) ** (1/(len(ret)*adj_period/252)) - 1
            SharpeRatio = (AReturnRate-rf)/AVolatility

            ret = ret.to_list()
            low_point = np.argmax((np.maximum.accumulate(ret)- ret)/np.maximum.accumulate(ret))
            if low_point == 0:
                MaxDrawdown = 0
            high_point = np.argmax(ret[:low_point])
            MaxDrawdown = (ret[high_point] - ret[low_point]) / ret[high_point]

            df_tmp = pd.DataFrame(data=[AReturnRate, AVolatility,
                                        SharpeRatio, MaxDrawdown,
                                        WinningRatio, PnLRatio],
                                   index=['AReturnRate', 'AVolatility',
                                          'SharpeRatio', 'MaxDrawdown',
                                          'WinningRatio', 'PnLRatio'],
                                  columns = [f'period{adj_period }'])

            summary_long_short_table = pd.concat([summary_long_short_table, df_tmp], axis=1)
        return summary_long_short_table

    def plot_daily_ret(self):
        fig = plt.figure(figsize=(10, 5*len(self.results)))
        for i in range(len(self.results)):
            adj_period = self.results[i]['adj_period']
            ret = self.results[i]['ret'].T.copy()
            ret = (ret.iloc[:, -1] - ret.iloc[:, 0]).to_frame()
            ret.columns = ['daily']
            ret = (100 * ret).round(2)
            ret['30d'] = ret['daily'].rolling(window=(1 + 20//adj_period)).mean()
            ax = fig.add_subplot(100*len(self.results) + 10 + (i+1))
            ax.bar(ret.index, ret['daily'], alpha=0.6, width=2)
            ax.plot(ret.index, ret['30d'], alpha=1, color='red')
            ax.legend(['30d', 'daily'], loc='upper left')  # 先写plot的，再写bar的
            adj_period = self.results[i]['adj_period']
            ax.set_title(f'Adj_periods_{adj_period}: Long Short Portfolio Returns', fontsize=15)
            ax.set_ylabel('return (%)')
        plt.tight_layout()
        plt.show()

    def fast_analysis(self,rating=False):
        self.calc_multiple_periods()
        print("\n{:-^120s}".format('Long Returns Performance'))
        self.plot_layer_ret_bar(excess=False, long=True)
        summary_layer_ret = self.summary_layer_ret(excess=False,long=True)
        print(summary_layer_ret)
        self.plot_layer_ret(excess=False, long=True)

        print("\n{:-^120s}".format('Short Returns Performance'))
        self.plot_layer_ret_bar(excess=False,long=False)
        summary_layer_ret = self.summary_layer_ret(excess=False,long=False)
        print(summary_layer_ret)
        self.plot_layer_ret(excess=False, long=False)

        print("\n{:-^120s}".format('Excess Long Returns Performance'))
        self.plot_layer_ret_bar(excess=True, long=True)
        summary_layer_ret = self.summary_layer_ret(excess=True,long=True)
        print(summary_layer_ret)
        self.plot_layer_ret(excess=True, long=True)

        print("{:=^120s}".format('IC'))
        summary_ic_table = self.summary_ic_table()
        print("\n{:-^120s}".format('IC performance'))
        print(summary_ic_table['ic'])
        print('\nGroup IC performance: ')
        print(summary_ic_table['group_ic'])
        print("\n{:-^120s}".format('IC yearly'))
        self.plot_ic_bar('Y', kind='rank_ic')
        print("\n{:-^120s}".format('IC monthly'))
        self.plot_monthly_ic(kind='rank_ic')
        print("\n{:-^120s}".format('IC daily'))
        self.plot_daily_ic(kind='rank_ic')
        print("\n{:-^120s}".format('Cumulative IC'))
        self.plot_cum_ic(kind='rank_ic')

# Alpha Construct and Analysis

The most obvious alpha is the correlation between volume and price

In [91]:
from scipy.stats import rankdata
def correlation(x, y, window=10):
    return x.rolling(window).corr(y)

def ts_rank(df, window=10):
    return df.rolling(window).apply(rolling_rank)

def ts_max(df, window=10):
    return df.rolling(window).max()

def rolling_rank(na):
    return rankdata(na)[-1]

def corr_volume_last(df_volume, df_last):
    df = correlation(ts_rank(df_volume, 5), ts_rank(df_last, 5), 5)
    df = df.replace([-np.inf, np.inf], 0).fillna(value=0)
    return -1 * ts_max(df, 3)

df_corr_volume_last = corr_volume_last(df_volume, df_last)